# AI-PSCI-017: Model Validation & Performance Metrics

**AI in Pharmaceutical Sciences: Bench to Bedside**  
VCU School of Pharmacy | VIP Program | Spring 2026

---

**Week 10 | Module: Testing & Evaluation | Estimated Time: 90-120 minutes**

**Prerequisites**: AI-PSCI-015 (AutoDock Vina), AI-PSCI-016 (DiffDock & GNINA)

---

## 🎯 Learning Objectives

After completing this talktorial, you will be able to:

1. **Systematically compare** docking and prediction methods using quantitative metrics
2. **Calculate and interpret RMSD** to assess pose prediction accuracy
3. **Analyze scoring correlations** between different docking methods
4. **Design validation experiments** with appropriate controls and statistics
5. **Make evidence-based recommendations** for method selection in drug discovery

---

## 📚 Background

### Why Validate Computational Methods?

In drug discovery, computational predictions guide expensive experimental decisions. A docking method that produces incorrect poses could lead to:
- Wasted synthesis efforts on compounds that don't actually bind
- Missed opportunities from discarding good candidates
- Incorrect structure-activity relationship (SAR) interpretations

**Validation** ensures our computational tools are fit for purpose before we rely on them for novel predictions.

### The Three Docking Methods

You've now used three different docking approaches:

| Method | Type | Key Feature | Scoring |
|--------|------|-------------|--------|
| **AutoDock Vina** | Physics-based | Fast, widely validated | kcal/mol (more negative = better) |
| **DiffDock** | AI (Diffusion) | Blind docking, no pocket needed | Confidence (0-1, higher = better) |
| **GNINA** | AI (CNN) | Neural network rescoring | CNN score (0-1) + affinity (pKd) |

### Key Validation Metrics

- **RMSD (Root Mean Square Deviation)**: Measures how closely a predicted pose matches the experimental structure. Lower is better.
  - < 2.0 Å: **Excellent** (successful pose prediction)
  - 2.0 - 4.0 Å: **Fair** (correct binding mode, some deviation)
  - > 4.0 Å: **Poor** (different binding mode or failed prediction)

- **Success Rate**: Percentage of predictions with RMSD < 2.0 Å

- **Scoring Correlation**: How well does the score predict experimental binding affinity?

### Key Concepts
- **RMSD**: Root Mean Square Deviation - geometric measure of pose accuracy
- **Cross-validation**: Testing on data not used for method development
- **Benchmarking**: Systematic comparison against known standards
- **Statistical significance**: Ensuring differences aren't due to chance

---

## 🛠️ Setup

Run this cell to install required packages:

In [ ]:
#@title 🛠️ Install Required Packages
!pip install rdkit biopython py3Dmol scipy -q
print("✅ Packages installed successfully!")

Import required libraries:

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# BioPython for structure analysis
from Bio.PDB import PDBParser, Superimposer
from Bio.PDB.PDBIO import PDBIO

# RDKit for molecular operations
from rdkit import Chem
from rdkit.Chem import AllChem, Draw

print("✅ Libraries imported successfully!")

---

## 🎯 Target Configuration

Select your drug target. This should match your selection from AI-PSCI-015 and AI-PSCI-016.

In [ ]:
#@title 🎯 Select Your Drug Target
TARGET = "DHFR" #@param ["DHFR", "ABL1", "EGFR", "AChE", "COX-2", "DPP-4"]

# Complete target configuration with all identifiers
TARGET_CONFIG = {
    "DHFR": {
        "pdb": "2W9S",
        "uniprot": "P0ABQ4",
        "chembl": "CHEMBL202",
        "drug": "Trimethoprim",
        "drug_smiles": "COc1cc(Cc2cnc(N)nc2N)cc(OC)c1OC",
        "ligand_code": "TOP",
        "expected_affinity": -8.5  # Approximate Ki ~0.5 nM
    },
    "ABL1": {
        "pdb": "1IEP",
        "uniprot": "P00519",
        "chembl": "CHEMBL1862",
        "drug": "Imatinib",
        "drug_smiles": "Cc1ccc(NC(=O)c2ccc(CN3CCN(C)CC3)cc2)cc1Nc1nccc(-c2cccnc2)n1",
        "ligand_code": "STI",
        "expected_affinity": -10.5  # Ki ~37 nM
    },
    "EGFR": {
        "pdb": "1M17",
        "uniprot": "P00533",
        "chembl": "CHEMBL203",
        "drug": "Erlotinib",
        "drug_smiles": "COCCOc1cc2ncnc(Nc3cccc(C#C)c3)c2cc1OCCOC",
        "ligand_code": "AQ4",
        "expected_affinity": -9.8  # Ki ~2 nM
    },
    "AChE": {
        "pdb": "4EY7",
        "uniprot": "P22303",
        "chembl": "CHEMBL220",
        "drug": "Donepezil",
        "drug_smiles": "COc1cc2CC(CC2cc1OC)CN1CCc2ccccc2C1=O",
        "ligand_code": "E20",
        "expected_affinity": -10.2  # Ki ~6 nM
    },
    "COX-2": {
        "pdb": "3LN1",
        "uniprot": "P35354",
        "chembl": "CHEMBL230",
        "drug": "Celecoxib",
        "drug_smiles": "Cc1ccc(-c2cc(C(F)(F)F)nn2-c2ccc(S(N)(=O)=O)cc2)cc1",
        "ligand_code": "CEL",
        "expected_affinity": -9.0  # Ki ~40 nM
    },
    "DPP-4": {
        "pdb": "1X70",
        "uniprot": "P27487",
        "chembl": "CHEMBL284",
        "drug": "Sitagliptin",
        "drug_smiles": "Fc1cc(F)c(C[C@H](N)CC(=O)N2CCn3c(nnc3C(F)(F)F)C2)c(F)c1F",
        "ligand_code": "715",
        "expected_affinity": -9.5  # Ki ~18 nM
    }
}

config = TARGET_CONFIG[TARGET]
print(f"✅ Target: {TARGET}")
print(f"   PDB: {config['pdb']} | UniProt: {config['uniprot']}")
print(f"   Reference Drug: {config['drug']}")
print(f"   Ligand Code in PDB: {config['ligand_code']}")
print(f"   Expected Affinity: {config['expected_affinity']} kcal/mol")

---

## 🔬 Guided Inquiry 1: Preparing Validation Data

### Context

Before we can compare methods, we need to organize our docking results. In AI-PSCI-015 and AI-PSCI-016, you generated poses using AutoDock Vina, DiffDock, and GNINA.

For this validation exercise, we'll simulate having those results and use the crystal structure as our ground truth. In a real workflow, you would load your actual docking output files.

### Your Task

Using your AI assistant, write code to:
1. Download the crystal structure for your target
2. Extract the bound ligand as the reference pose
3. Create a DataFrame to organize docking results from all three methods
4. Include scores, RMSD values, and method metadata

💡 **Prompting Tips**:
- Ask: "How do I extract a ligand from a PDB file using BioPython?"
- If you have actual docking output files, ask how to parse SDF or PDBQT files
- Request help organizing multi-method comparison data

### Verification
After running your code, confirm:
- [ ] Crystal structure downloaded successfully
- [ ] Reference ligand extracted with correct number of atoms
- [ ] DataFrame created with columns for each method's scores and RMSD

📓 **Lab Notebook**: Document the source of your validation data and any preprocessing steps applied.

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 2: RMSD Analysis

### Context

RMSD (Root Mean Square Deviation) is the gold standard for assessing docking pose accuracy. It measures the average distance between corresponding atoms in two structures.

**RMSD Formula**:
$$RMSD = \sqrt{\frac{1}{N} \sum_{i=1}^{N} |r_i^{pred} - r_i^{ref}|^2}$$

Where:
- $N$ = number of atoms
- $r_i^{pred}$ = predicted position of atom $i$
- $r_i^{ref}$ = reference (crystal) position of atom $i$

### Your Task

Using your AI assistant, write code to:
1. Visualize the RMSD distribution for each docking method
2. Calculate summary statistics (mean, median, std) for each method
3. Determine the success rate (RMSD < 2.0 Å) for each method
4. Create a box plot comparing RMSD across methods

💡 **Prompting Tips**:
- Ask: "How do I create side-by-side box plots with matplotlib?"
- Request help adding a horizontal line at the 2.0 Å threshold
- Ask about statistical tests to compare distributions

### Verification
After running your code, confirm:
- [ ] Box plot shows clear comparison between all three methods
- [ ] 2.0 Å threshold line is visible
- [ ] Success rates calculated for each method
- [ ] Summary statistics table generated

📓 **Lab Notebook**: Record which method has the highest success rate. Is this consistent with the published literature?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 3: Scoring Correlation Analysis

### Context

A good docking method should not only predict correct poses but also rank them correctly. The best pose should have the best score. We assess this by examining:

1. **Score-RMSD correlation**: Do better scores correlate with lower RMSD?
2. **Method agreement**: Do different methods agree on what's a good pose?
3. **Rank correlation**: Do methods agree on the rank order?

### Your Task

Using your AI assistant, write code to:
1. Create scatter plots of Score vs RMSD for each method
2. Calculate Pearson and Spearman correlations
3. Analyze whether the top-ranked pose is also the best RMSD
4. Compare rank order agreement between methods

💡 **Prompting Tips**:
- Ask: "What's the difference between Pearson and Spearman correlation?"
- Note that Vina scores are negative (more negative = better) while CNN scores are positive (higher = better)
- Request help with correlation coefficient interpretation

### Verification
After running your code, confirm:
- [ ] Scatter plots show Score vs RMSD for each method
- [ ] Correlation coefficients calculated and interpreted
- [ ] Top-1 accuracy reported (is best-scored pose also best RMSD?)

📓 **Lab Notebook**: Which method shows the best score-RMSD correlation? What does this mean for virtual screening?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 4: Statistical Method Comparison

### Context

When comparing methods, we need to determine if observed differences are statistically significant or could have occurred by chance. Common tests include:

- **Paired t-test**: Compare two methods on same test cases
- **Wilcoxon signed-rank test**: Non-parametric alternative
- **ANOVA**: Compare multiple methods simultaneously

### Your Task

Using your AI assistant, write code to:
1. Perform statistical tests comparing RMSD distributions between methods
2. Calculate 95% confidence intervals for mean RMSD
3. Create a visualization showing statistical significance
4. Report p-values with appropriate interpretation

💡 **Prompting Tips**:
- Ask: "How do I perform a paired t-test in Python with scipy?"
- Ask about assumptions for parametric vs non-parametric tests
- Request help with multiple comparison correction (e.g., Bonferroni)

### Verification
After running your code, confirm:
- [ ] Pairwise comparisons performed for all method pairs
- [ ] P-values reported with significance interpretation
- [ ] Confidence intervals calculated
- [ ] Visualization clearly shows significant differences

📓 **Lab Notebook**: Are the differences between methods statistically significant? What sample size would you need for more confident conclusions?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 5: Method Performance Dashboard

### Context

A comprehensive performance dashboard helps stakeholders quickly assess method capabilities. This is essential for:
- Choosing methods for production workflows
- Communicating results to medicinal chemists
- Documenting method validation in publications

### Your Task

Using your AI assistant, write code to:
1. Create a multi-panel dashboard comparing all metrics
2. Include radar/spider chart for method comparison
3. Add a summary table with all key metrics
4. Generate a single publication-quality figure

💡 **Prompting Tips**:
- Ask: "How do I create a radar chart in matplotlib?"
- Request help with subplot layout for complex figures
- Ask about color-blind friendly palettes

### Verification
After running your code, confirm:
- [ ] Dashboard contains multiple informative panels
- [ ] Radar chart shows relative method strengths
- [ ] All metrics are clearly labeled
- [ ] Figure saved at publication quality (300 dpi)

📓 **Lab Notebook**: What are the key takeaways from your dashboard? Which method would you recommend for your target?

In [ ]:
# Your code here



---

## 🔬 Guided Inquiry 6: Evidence-Based Recommendations

### Context

The ultimate goal of validation is to make informed decisions. Based on your analysis, you should be able to:
- Recommend the best method for your specific application
- Justify the recommendation with evidence
- Identify scenarios where different methods might be preferred

### Your Task

Using your AI assistant, write code to:
1. Create a decision framework based on your validation results
2. Generate a recommendation report with supporting evidence
3. Include caveats and limitations
4. Suggest next steps for production use

💡 **Prompting Tips**:
- Ask: "How do I create a formatted markdown report in Python?"
- Think about different use cases: virtual screening vs. lead optimization
- Consider computational cost vs. accuracy tradeoffs

### Verification
After running your code, confirm:
- [ ] Clear recommendation with primary and secondary choices
- [ ] Evidence cited for each recommendation
- [ ] Limitations and caveats documented
- [ ] Report saved as markdown file

📓 **Lab Notebook**: What additional validation would you perform before using these methods in a real drug discovery project?

In [ ]:
# Your code here



---

## ✅ Checkpoint

Before moving on to the next talktorial, confirm you can:

- [ ] Calculate RMSD between predicted and reference poses
- [ ] Interpret RMSD thresholds (< 2.0 Å = success)
- [ ] Perform statistical comparisons between methods
- [ ] Create publication-quality validation figures
- [ ] Generate evidence-based method recommendations
- [ ] Document limitations and caveats honestly

### Your lab notebook should include:
- [ ] RMSD summary statistics for each method
- [ ] Statistical test results and p-values
- [ ] Your method recommendation with supporting evidence
- [ ] Discussion of limitations and next steps

---

## 🤔 Reflection Questions

Answer these in your lab notebook:

1. **Method Selection**: If you could only use ONE docking method for your research project, which would you choose and why? Consider accuracy, speed, and reliability.

2. **Validation Strategy**: What additional experiments would you perform to increase confidence in your method selection before using it for novel compound predictions?

3. **Limitations**: What are the potential pitfalls of relying solely on RMSD for docking validation? What other metrics might be informative?

4. **Translation to Practice**: How would you communicate these validation results to a medicinal chemistry team who will use the predictions to guide synthesis decisions?

---

## 📖 Further Reading

- [Warren et al. (2006) - A Critical Assessment of Docking Programs and Scoring Functions](https://doi.org/10.1021/jm050362n) - Classic benchmarking study
- [Bender & Cortés-Ciriano (2021) - Artificial Intelligence in Drug Discovery](https://doi.org/10.1038/s41573-021-00290-8) - Review of AI methods including DiffDock
- [McNutt et al. (2021) - GNINA 1.0: Molecular Docking with Deep Learning](https://doi.org/10.26434/chemrxiv.13578635) - GNINA methodology paper
- [Corso et al. (2023) - DiffDock: Diffusion Steps, Twists, and Turns for Molecular Docking](https://arxiv.org/abs/2210.01776) - DiffDock methodology
- [Cross et al. (2009) - Comparison of Several Molecular Docking Programs](https://doi.org/10.1021/ci800324m) - Comprehensive docking comparison

---

## 🔗 Connection to Research

Method validation is a critical but often underappreciated step in computational drug discovery. The skills you've developed here directly translate to:

**Academic Research**:
- Benchmarking new algorithms against established methods
- Publishing validation studies for novel targets
- Supporting computational predictions with statistical evidence

**Industry Applications**:
- Qualifying computational methods for production use
- Satisfying regulatory requirements for in silico evidence
- Making resource allocation decisions (which method to run at scale)

**Your Target**:
Having validated docking methods for your specific target, you now have:
- Confidence in which method to use for novel compound predictions
- Understanding of expected accuracy and failure modes
- Documentation to support your computational approach

This validation framework can be applied to any computational method in drug discovery: ADMET prediction, binding affinity scoring, protein structure prediction, and more.

---

*AI-PSCI-017 Complete. Proceed to AI-PSCI-018: Debugging & Optimization Strategies.*